# Objectives 

- Familiarization with smart contarcts 
- Step through OPCODE level execution of a transaction
- Familiarize and set up web3 library and wallet APIs 
- Interacting with an ERC20 token
- Interacting with a contract using Etherscan
- Importing ERC20 tokens on Metamask
- Interacting atomically with two Dapps



# Walkthrough smart contract

We have deployed smart contracts for the ERC20 tokens on the Sepolia testnet. The contracts can be found on etherscan here: [USTHKD](https://sepolia.etherscan.io/address/0x64b6f44862e8800ebd63cd7f1319c8bf0bc1bb99), [USTETH](https://sepolia.etherscan.io/address/0xd10a9ec6e827b13cf399fcc7e6424f233e99562a). Let us now go through the contract and describe what each ```function``` does.

### What does ```balanceOf``` and ```totalSupply``` do?

### What does ```transfer``` and ```transferFrom``` do ? Which of them needs you to run ```approve``` before it executes correctly ?

### What events can be emitted by the contract ? [What is their use](https://consensys.net/blog/developers/guide-to-events-and-logs-in-ethereum-smart-contracts/) ? 

### How does the contract respond to airdrop requests ?

# Requesting tokens

To request the tokens we shall be using in the future for some in-class labs, we are going to use the ```web3``` library in python. The installation instructions can be found [here](https://web3py.readthedocs.io/en/v5/quickstart.html). We will view our transactions on the [Sepolia testnet etherscan](https://sepolia.etherscan.io/). 

We now send a transaction that uses this API to request some of the Lab ERC20 tokens from a [faucet contract](https://sepolia.etherscan.io/address/0x1bc75ae4fbd3e100717bb15661764b13667ef7ac). The faucet allows each registered wallet to redeem a fixed number of tokens per day. The code used to do that has to be written below.

### Setup Web3 API
First, we connect your wallet to a Web3 API - this is a service that connects you to the Ethereum blockchain via a simple URL. 

To do that, first create a free account on [Infura](https://app.infura.io/register)(using any email ID).

Once you log in, you would see your Infura API key under the project name you had given. Use that key in the following code.

In [1]:
# Import web3
from web3 import Web3
from datetime import datetime, timedelta

w3 = Web3(Web3.HTTPProvider('https://sepolia.infura.io/v3/your-infura-key'))

print(w3.eth.block_number) #Check if the notebook is connected to the Infura node
print(w3.__dict__)

5013273
{'manager': <web3.manager.RequestManager object at 0x000002122CACB5D0>, 'codec': <eth_abi.codec.ABICodec object at 0x000002122F883F50>, 'eth': <web3.eth.eth.Eth object at 0x000002122CAA2010>, 'net': <web3.net.Net object at 0x000002122CAC9B50>, 'geth': <web3.geth.Geth object at 0x000002122CAB0FD0>, 'tracing': <web3.tracing.Tracing object at 0x000002122F881A90>, 'testing': <web3.testing.Testing object at 0x000002122F881990>, '_ens': <web3._utils.empty.Empty object at 0x000002122CF6CE90>}


### Setup your ETH account details

In [2]:
 # Wallet details

public_address = Web3.to_checksum_address('your-sepolia-address') # see metamask for this
private_key = 'your-private-key' # see metamask for this

### Set up a contract instance
Now, we send a request transaction to the faucet, signed by your private key.

In [20]:
# Define the USTFaucet contract to connect to

contract_addr = Web3.to_checksum_address('faucet-contract-address')
abi_contract = '[contract-abi-info]'
contract = w3.eth.contract(contract_addr, abi=abi_contract)

# Define the address of the ERC20 token you want

token_addr = Web3.to_checksum_address('the-token-address')

### Create transaction data

In [21]:
### Create transaction data
txn = contract.functions.requestTokens(token_addr).build_transaction(
    {
        'nonce':w3.eth.get_transaction_count(public_address),
        'from':public_address,
    }
)

### Sign and post transaction
You should see the hash of your transaction printed.

In [22]:
# Post transaction

signed_txn = w3.eth.account.sign_transaction(txn, private_key) # Sign the txn with your private key 
txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction) # Send it onto the chain
print("Hash of swap transaction : ",w3.to_hex(txn_hash)) # Search the txn hash on etherscan to see if it got confirmed

Hash of swap transaction :  0xc05f0bbe50051d4e006861d72ff9c80fc37a4c353d649d6271f27f0a39e879c6


## Alternate: Request tokens through Etherscan

Verified contracts on Etherscan allow you to interact with functions on the contract.
Open the [USTFaucet](https://sepolia.etherscan.io/address/0x1bc75ae4fbd3e100717bb15661764b13667ef7ac) contract by providing it's address. You can now access the contract's read and write functions.
Go to the ``write tab``, connect your metamask wallet and call the ``requestTokens`` function by providing your wallet's public address and the USTETH token's contract address. Confirm the transaciton on the metamask prompt.

# Add to MetaMask

- The token balances can now be tracked via MetaMask using the contract addresses of each token. 
- Use the "Import Tokens" link at the bottom of your MetaMask. 
- Enter the smart contract address for an ERC20 tokens we would be using in the class.
- Do this for each of the tokens - USTETH and USTHKD




# Interacting atomically with two Dapps


## Hotel and Flight contracts

Assume you are going on a vacation and need to book travel and accommodation. We have created two contracts mimicking a decentralized application for booking accommodation and travel. The hotel reservation contract can be found [here](https://sepolia.etherscan.io/address/0x65A8aa6334A500Da1dA420Bc5E965c55E3BD7f21). Let us go through its main ``bookRoom()`` function

- Initial few checks ensure the payment is sufficient and rooms are available at the hotel
- The state of the contract is updated to make sure the room is booked in the name of the wallet initiating the transaction calling the function
- The last part calculates and disburses a refund if the transaction caller sent more money than the ticket price of `10**10 wei`.
- Note that no part of the contract allows for cancellation of the reservation; this Dapp corresponds to a Non-refundable reservation

You will now book accommodation for the trip. 
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. - - Click the [bookRoom()](https://sepolia.etherscan.io/address/0x65A8aa6334A500Da1dA420Bc5E965c55E3BD7f21#writeContract#F2) tab. 
- Set the amount of Wei to send as a payment. The required payment is `10000000000 wei` or `0.00000001 ETH` (recommend); you can send more if you want, and the contract will refund it within the same transaction.
- Check the status of your reservation by querying `CheckReservations` with your public key

Now that we have booked the accommodation, let us book a flight. The flight reservation contract looks similar to the hotel reservation contract with ``bookFlight()`` replacing ``bookRoom()``. You can access the flight reservation contract [here](https://sepolia.etherscan.io/address/0x27dD8de8c733c546FeEa68bE6CeFBFFb7Bb8A219). Perform the same steps as the hotel reservation contract to book a flight ticket. Check the status of the transaction on Etherscan.

Notice that the transaciton will fail for some of you since there aren't enough flight tickets available for all class members. This leads to a situation where your hotel reservation is practically useless since there is no way to travel. Is it possible to avoid such a situation: *Book hotel only if the flight is available* and *Book flight only if the hotel is available*? In other words, we want the trip reservation to be atomic - Either book both travel and accommodation or book none. Notice that the Dapps are independent of one another - think of them as two separate travel websites. 

## Atomic transaction

We can perform such transactions easily on blockchains. Note that an ethereum transaction either succeeds or fails and reverts back. Thus, we need to combine ``bookRoom()`` and ``bookFlight()`` into a single transaction. 

We can do this by launching a smart contract whose one function call will call two of the above hotel and flight contracts. The atomic transaction contract (aka trip reservation contract) can be found [here](https://sepolia.etherscan.io/address/0xa67990332E026b6c4AC71937BbE44207d97c113a). Let us quickly go through the ``bookTrip`` function:
- The function takes in 4 arguments - the addresses of Hotel and Flight reservation contracts and their prices
- It then calls both the hotel and flight contract 
- The calls are sequential, however, due to the atomic nature of an Ethereum transaction, the whole ``bookTrip`` transaction will fail if either of the contract calls fails.
- The last part disburses any remaining refund.

I have refreshed both the hotel and flight contracts, and you will now use the atomic transaction contract to book both atomically:
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. - - Click the [bookTrip](https://sepolia.etherscan.io/address/0xa67990332E026b6c4AC71937BbE44207d97c113a#writeContract#F1) tab. 
- Set the amount of ETH to send as a payment. The required total payment is `20000000000 wei` or `0.00000002 ETH`; anything extra will be refunded. 
- Give the contract addresses of the hotel and flight reservation contract; set the price of both to ``10000000000`` wei.
- Check the status of the transaction on Etherscan

The transaction will succeed for some of you and fail for the rest. Check the status of your booking on the respective contracts. If the atomic transaction failed, you should not have confirmed reservations on both the hotel and flight contracts. 